In [ ]:
# !uv pip install google-cloud-aiplatform

In [1]:
# syft absolute
import syft as sy

In [2]:
node = sy.orchestra.launch(
    name="test-domain-1",
    port="auto",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
)
domain_client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Starting test-domain-1 server on 0.0.0.0:58450
Waiting for server to start.

INFO:     Started server process [83185]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:58450 (Press CTRL+C to quit)


.
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=1b0ab9501a194c95afe934e1e1bb3a83 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
INFO:     127.0.0.1:52286 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.
INFO:     127.0.0.1:52288 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:52288 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:52288 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:52288 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:52290 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# Admin

In [3]:
# !curl localhost:9585/api/v2/metadata

In [4]:
# domain_client = sy.login(url="http://34.96.72.102", email="info@openmined.org", password="changethis")

In [5]:
# domain_client

In [6]:
# stdlib
import os

# PRIVATE_KEY = os.environ["BEACH_PRIVATE_KEY"].replace("\\n", "\n")

In [9]:
# @sy.api_endpoint(
#     path="vertex.run"
# )
# def public_endpoint_method(
#     context,
#     prompt: str,
# ) -> str:
#     print("got prompt", prompt)
#     return {"prediction": "test"}

In [10]:
def allowed(state, email: str) -> bool:
    return False

In [11]:
models = {"gemma-2b": "3213239169291649024"}

In [41]:
@sy.api_endpoint(
    path="vertex.run",
    settings={"SERVICE_ACCOUNT": SERVICE_ACCOUNT},
)
def public_endpoint_method(
    context,
    prompt: str,
    max_tokens: int = 50,
    temperature: float = 0.1,
    top_p: float = 1.0,
    top_k: int = 1,
    raw_response: bool = False,
) -> str:
    print(prompt)
    print(max_tokens)
    print(temperature)
    print(top_p)
    print(top_k)
    print(raw_response)
    # context.state = ["jimmy@caltech.edu": {"n_calls":3, "period_secs":60}]

    # use helper function rate limiter
    # is_allowed = allowed(context.state, context.user.email)

    # third party
    from google.cloud import aiplatform
    from google.oauth2 import service_account

    try:
        credentials = service_account.Credentials.from_service_account_info(
            context.settings["SERVICE_ACCOUNT"]
        )

        PROJECT_ID = "project-enigma-415021"
        REGION = "us-west1"
        ENDPOINT_ID = "3213239169291649024"
        aip_endpoint_name = (
            f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}"
        )
        endpoint_vllm = aiplatform.Endpoint(aip_endpoint_name, credentials=credentials)
        default_kwargs = {
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "top_k": top_k,
            "raw_response": raw_response,
        }
        instances = [
            default_kwargs,
        ]
        response = endpoint_vllm.predict(instances=instances)
        prediction = response.predictions[0]
    except Exception as e:
        prediction = f"Error: Please try again? {e}"
    return {"prediction": prediction}

In [27]:
# domain_client.refresh()

In [28]:
# new_endpoint = sy.TwinAPIEndpoint(
#     path="vertex.run",
#     private_function=private_run_vertex,
#     mock_function=mock_run_vertex,
#     description="Run vertex model",
# )
# new_endpoint

In [29]:
# domain_client.user_role

In [30]:
# use to delete if you want to add again (there is a seperate update API)
domain_client.api.services.api.delete(endpoint_path="vertex.run")

INFO:     127.0.0.1:52310 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully deleted.

In [31]:
domain_client.api.services.api.add(endpoint=public_endpoint_method)

INFO:     127.0.0.1:52312 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully created.

## Create Data Scientist

In [32]:
domain_client.register(
    name="Jimmy Doe",
    email="jimmy@caltech.edu",
    password="abc123",
    password_verify="abc123",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

INFO:     127.0.0.1:52314 - "POST /api/v2/register HTTP/1.1" 200 OK


SyftError: User already exists with email: jimmy@caltech.edu

# Data Scientist

In [33]:
# users = domain_client.api.services.user.search(name="Jimmy Doe")
# user = users[0]
# user.mock_execution_permission

In [34]:
# user.update(mock_execution_permission=True)

In [35]:
# users = domain_client.api.services.user.search(name="Jimmy Doe")
# user = users[0]
# user.mock_execution_permission

In [36]:
# jimmy_client = sy.login(url="http://34.96.72.102", email="jimmy@caltech.edu", password="abc123")

In [37]:
jimmy_client = node.login(email="jimmy@caltech.edu", password="abc123")

INFO:     127.0.0.1:52317 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:52317 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:52317 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:52317 - "GET /api/v2/api?verify_key=8cc3567d95d071df2279db611dd507c60db97caa85bc7cbc048a3af004fba3f9&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:52319 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <jimmy@caltech.edu>


In [38]:
jimmy_client.api.services.api.api_endpoints()

INFO:     127.0.0.1:52321 - "POST /api/v2/api_call HTTP/1.1" 200 OK


[syft.service.api.api.TwinAPIEndpointView]

In [39]:
jimmy_client.api.services.vertex.run(prompt="where am i")

INFO:     127.0.0.1:52323 - "POST /api/v2/api_call HTTP/1.1" 200 OK


{'prediction': "Prompt:\nwhere am i\nOutput:\n?\n\nI'm in the middle of a 1000 mile road trip and I'm wondering if I'm in the right place.\n\nI'm in the middle of a 1000 mile road trip and"}

Logged into <test-domain-1: High side Domain > as GUEST
where am i
50
0.1
1.0
1
False


In [ ]:
assert False

In [ ]:
# jimmy_client.api.services.vertex.run(prompt="test", raw_response=True)

## Create Input Policy

In [ ]:
domain_client.refresh()

In [ ]:
domain_client.api.services.vertex.run.private(prompt="hello")

In [ ]:
input_policy = sy.MixedInputPolicy(
    func=jimmy_client.api.services.vertex.run,
    prompt=str,
    max_tokens=int,
    temperature=float,
    top_p=float,
    top_k=int,
    raw_response=bool,
)
input_policy

## Create Output Policy

In [ ]:
class RateLimiter(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: list[str] = []
    state: dict = {}

    def __init__(self, n_calls=1, downloadable_output_args: list[str] = None):
        self.downloadable_output_args = (
            downloadable_output_args if downloadable_output_args is not None else []
        )
        self.n_calls = n_calls
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]

    def update_policy(self, context, outputs):
        self.state["counts"] += 1

    def apply_to_output(self, context, outputs, update_policy=True):
        if hasattr(outputs, "syft_action_data"):
            outputs = outputs.syft_action_data
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]
            if update_policy:
                self.update_policy(context, outputs)
        else:
            return "You've hit the rate limit. Please contact the administrator."

        output_dict["calls_remaining"] = self.n_calls - self.state["counts"]
        return output_dict

    def _is_valid(self, context):
        return self.state["counts"] < self.n_calls

In [ ]:
@sy.syft_function(
    input_policy=input_policy,
    output_policy=RateLimiter(n_calls=3, downloadable_output_args=["prediction"]),
)
def my_vertex_func(
    func,
    prompt: str,
    max_tokens: int = 50,
    temperature: float = 0.1,
    top_p: float = 1.0,
    top_k: int = 1,
    raw_response: bool = False,
):
    return func(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        raw_response=raw_response,
    )

In [ ]:
my_vertex_func

In [ ]:
## TODO: Local Execution doesn't work

In [ ]:
# @sy.syft_function()
#     input_policy=input_policy,
#     output_policy=RateLimiter(n_calls=3, downloadable_output_args=["prediction"]),
# )
# def my_vertex_func(
#     func,
#     prompt: str,
#     max_tokens: int,
#     temperature: float,
#     top_p: float,
#     top_k: int,
#     raw_response: bool
# ):
#     return func(
#         prompt=prompt,
#         max_tokens=max_tokens,
#         temperature=temperature,
#         top_p=top_p,
#         top_k=top_k,
#         raw_response=raw_response
#     )

In [ ]:
# my_vertex_func(func=jimmy_client.api.services.vertex.run, prompt="my prompt")

In [ ]:
## TODO: this would be handy

In [ ]:
# def test_func(*args, **kwargs):
#     print(kwargs)
#     return {"prediction": f"Test API: {kwargs['prompt']}"}

In [ ]:
# my_vertex_func(func=test_func, prompt="my prompt")

In [ ]:
# assert False

In [ ]:
new_project = sy.Project(
    name="Vertex Model Access",
    description="Hi, I want to use this model 3 times",
    members=[jimmy_client],
)

new_project.create_code_request(my_vertex_func, jimmy_client)

## Admin approves

In [ ]:
domain_client.requests

In [ ]:
request = domain_client.requests[-1]
request

In [ ]:
request.approve()

## Data Scientist runs

In [ ]:
jimmy_client.refresh()

In [ ]:
jimmy_client.code.my_vertex_func

In [ ]:
jimmy_client.api.services.vertex.run.custom_function_actionobject_id()

In [ ]:
result = jimmy_client.code.my_vertex_func(
    func=jimmy_client.api.services.vertex.run,
    prompt="Who are you now?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result

In [ ]:
result = jimmy_client.code.my_vertex_func(
    func=jimmy_client.api.services.vertex.run,
    prompt="Who are you now?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result

In [ ]:
result = jimmy_client.code.my_vertex_func(
    func=jimmy_client.api.services.vertex.run,
    prompt="Who are you now?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result

In [ ]:
result = jimmy_client.code.my_vertex_func(
    func=jimmy_client.api.services.vertex.run,
    prompt="Who are you now?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result

## Advanced

In [ ]:
# assert False

In [ ]:
# Allow api endpoint code to generate a policy and code submission object

In [ ]:
domain_client.refresh()

In [ ]:
@sy.private_api_endpoint()
def private_user_function_creator(
    context, func_name: str, limit: int, reason: str
) -> str:
    # syft absolute
    import syft as sy

    view_api = context.node.get_service_method("apiservice.view")
    api = view_api(context=context, path="vertex.run")

    # create input policy
    input_policy = sy.MixedInputPolicy(
        func=api.action_object_id,
        prompt=str,
        max_tokens=int,
        temperature=float,
        top_p=float,
        top_k=int,
        raw_response=bool,
    )

    output_policy = sy.OutputPolicyExecuteCount(limit=limit)

    my_vertex_func = sy.syft_function_string(
        input_policy=input_policy,
        output_policy=output_policy,
        raw_code=f"""
def {func_name}(
    func,
    prompt: str,
    max_tokens: int = 50,
    temperature: float = 0.1,
    top_p: float = 1.0,
    top_k: int = 1,
    raw_response: bool = False,
):
    return func(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        raw_response=raw_response,
    )
""",
    )

    new_project = sy.Project(
        name="Vertex Model Access",
        description=reason,
        members=[context.client],
    )

    result = new_project.create_code_request(my_vertex_func, context.client)
    return result

In [ ]:
creator_endpoint = sy.TwinAPIEndpoint(
    path="vertex.create_code_request",
    private_function=private_user_function_creator,
    mock_function=private_user_function_creator.public(),
    description="Create a vertex code request",
)
creator_endpoint

In [ ]:
response = domain_client.api.services.api.delete(
    endpoint_path="vertex.create_code_request"
)
response

In [ ]:
response = domain_client.api.services.api.add(endpoint=creator_endpoint)
response

In [ ]:
# domain_client.refresh()

In [ ]:
# domain_client.api.services.vertex.create_code_request(func_name="test", limit=2, reason="I want to run Vertex")

In [ ]:
jimmy_client.refresh()

In [ ]:
jimmy_client.api.services.vertex.create_code_request(
    func_name="test", limit=2, reason="I want to run Vertex"
)

In [ ]:
domain_client.requests

In [ ]:
domain_client.requests[0].approve()

In [ ]:
# we need to make the RemoteFunction serializable andchange what comes back to the user side from the Admin API
# or add a context.submit as user

In [ ]:
jimmy_client.refresh()

In [ ]:
# jimmy_client.code.test()

In [ ]:
# assert False

In [ ]:
result = jimmy_client.code.test(
    func=jimmy_client.api.services.vertex.run,
    prompt="How cool is OpenMined!!!?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result

In [ ]:
result = jimmy_client.code.test(
    func=jimmy_client.api.services.vertex.run,
    prompt="How cool is OpenMined!!!?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result

In [ ]:
result = jimmy_client.code.test(
    func=jimmy_client.api.services.vertex.run,
    prompt="How cool is OpenMined!!!?",
    max_tokens=50,
    temperature=0.1,
    top_p=1.0,
    top_k=1,
    raw_response=False,
)
result